In [1]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

os.chdir('../')
from scripts import hyperSN_model, dataloader

In [2]:
model_chkpt = './models/hyperSN/gl9tvseq/best_model-v1.ckpt'
model = hyperSN_model.HyperSN.load_from_checkpoint(model_chkpt)
model.eval()

C:\Users\stude\anaconda3\envs\biocycling_px\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:55: The loaded checkpoint was produced with Lightning v2.2.0, which is newer than your current Lightning version: v2.1.3


HyperSN(
  (conv1): Sequential(
    (0): Conv3d(1, 8, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv3d(8, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv3): Sequential(
    (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (dense1): Sequential(
    (0): Linear(in_features=20736, out_features=254, bias=True)
   

In [8]:
path_val = "F:/bcd_val_sub/data"

dataloader = dataloader.HyperspectralDataset(path_data=path_val, 
                                             window_size=model.hparams.patch_size,
                                             stride=1,
                                             in_channels=model.hparams.in_channels,
                                             mode='test',
                                             sample_strategy='grid')

In [9]:
iterator = dataloader.iterate_full_cube()

In [10]:
window, mask, cube_index, i, j = next(iterator)

Loading cube 0 from E0057


In [11]:
img_result = np.zeros(dataloader.current_cube.shape[:2])
while dataloader.current_cube_index == 0:
    window, mask, cube_index, i, j = next(iterator)
    
    # window to float tensor
    window = torch.from_numpy(window)
    window = window.unsqueeze(0).float().to(model.device)
    
    result = model.forward_pass(window)
    result = result.squeeze(0).detach().numpy()
    img_result[i:i+model.hparams.patch_size, j:j+model.hparams.patch_size] = result
    
    print(i, j)
    break

ValueError: expected 5D input (got 4D input)

In [ ]:
window.shape